#　ダッシュボード用データ作成(4scale)

## 使い方

基本的には上から実行  
ただし、評価者情報を出さない場合（素点がない場合）は分岐

## 必要データ
dataフォルダに配置
- 補正後スコア(csv)
- 過去補正後スコア(csv, ある場合)
- 今回測定者ユーザー情報(csv)
    - 必要情報
        - 必須 : ['GROW ID', '社員番号', '氏名']
        - あれば(なくても動く): ['GROW ID', '社員番号', '氏名']
- 評価者も含めたユーザー情報(csv)
    - 必要情報
      - 必須 : ['GROW ID', '社員番号', '氏名']
- 素点(csv):
    - 必要情報: ['EVALUATEE_ID', 'EVALUATOR_ID', 'COMPETENCY_NAME', 'SCORE', 'ANSWER_TIME', 'RELATION_NAME']
    - カラム名は事前に合わせておく
- 過去素点(csv):　必要カラム名は上記と同じ

## ライブラリインポート

In [1]:
import polars as pl
import numpy as np
import math
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import joblib

In [16]:
#自己評価suffix
self_suffix = "（自己評価）"

In [17]:
cog = ['課題設定', '解決意向','創造性','論理的思考', '疑う力',]
cog_self = [f"{val}{self_suffix}" for val in cog]
sel = ['個人的実行力', '内的価値', 'ヴィジョン',
        '自己効力', '成長', '興味', '耐性',  '感情コントロール', '決断力']
sel_self = [f"{val}{self_suffix}" for val in sel]
oth = ['表現力', '共感・傾聴力', '外交性', '柔軟性', '寛容', '影響力の行使', '情熱・宣教力']
oth_self = [f"{val}{self_suffix}" for val in oth]
com = ['組織への働きかけ', '地球市民','組織へのコミットメント',  '誠実さ']
com_self = [f"{val}{self_suffix}" for val in com]

cat = ["認知領域平均","自己領域平均","他者領域平均","コミュニティー領域平均", "全コンピテンシー平均",]
bn = ['外向性-内向性', '開放性-保守性', '繊細性-平穏性', '協調性-独立性', '自律性-自由性']
cn = cog+sel+oth+com
cn_self = cog_self+sel_self+oth_self+com_self
all_items = cog + ["認知領域平均"] + sel + ["自己領域平均"] + oth + ["他者領域平均"] + com + ["コミュニティー領域平均"] + ["全コンピテンシー平均"]
all_items_self = cog_self + [f"認知領域平均{self_suffix}"] +\
                    sel_self + [f"自己領域平均{self_suffix}"] + \
                    oth_self + [f"他者領域平均{self_suffix}"] + \
                    com_self + [f"コミュニティー領域平均{self_suffix}"] + [f"全コンピテンシー平均{self_suffix}"]


## ファイルパス

In [18]:
# ベンチマークPATH
benchmark_path = "./data/20250922_BigData_上司1同僚1部下1_3人強制変換_extrapolated.xlsx"

In [19]:
# 補正後データPATH
## 今回受験者データpath
path = "sample/補正済みスコア_sample_1107.csv"
##　　前回受験者データpath(あれば)
path_prev1 = "sample/補正済みスコア(前回)_sample_1107.csv"
##　　前々回受験者データpath(あれば)
path_prev2 = None

## ユーザーリスト・属性情報データPATH
#今回受験者属性情報(必要に応じて書き換えてください)
path_info = "sample/マスターデータ_sample_今回受験者_1107.csv"
# 評価者も含めた　ユーザーリスト
path_info_all = "sample/マスターデータ_sample.csv"

## 素点
# 今回測定ファイルパス
soten_path = "sample/評点データ_sample_1107.csv"
#過去測定ファイルパスリスト
prev_soten_paths = ["sample/評点データ_前回_sample_1107.csv"]
# 初測定の場合
# prev_soten_paths = None

## モデル
pca_path = "model/pca_model_4scale.pkl"
umap_path = "model/umap_model_canberra_4scale.pkl"
kmeans_path = "model/kmeans_4scale.pkl"

In [20]:
#保存するファイル名
filename = "ダッシュボード用データ_4scale.xlsx"

## ベンチマーク

In [21]:
benchmark_org = pl.read_excel(benchmark_path, infer_schema_length=10000)
benchmark = benchmark_org.select(cn)

In [22]:
benchmark

課題設定,解決意向,創造性,論理的思考,疑う力,個人的実行力,内的価値,ヴィジョン,自己効力,成長,興味,耐性,感情コントロール,決断力,表現力,共感・傾聴力,外交性,柔軟性,寛容,影響力の行使,情熱・宣教力,組織への働きかけ,地球市民,組織へのコミットメント,誠実さ
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
85.805724,80.714221,52.787684,82.710388,81.803923,80.714221,62.742922,59.231572,74.343905,75.477645,73.022021,70.008729,73.022021,75.403008,66.474531,70.362586,60.991463,67.651824,71.165129,61.961236,67.421795,80.519923,59.033855,80.261563,65.649942
48.43443,46.428576,56.514823,37.699275,49.745994,68.442048,67.937952,60.511839,59.734977,61.764349,52.35524,64.800496,73.844882,62.253919,55.160429,78.915463,78.376028,69.795949,77.004937,66.7038,70.007737,69.223895,51.067176,58.048505,63.526382
70.758537,51.144055,53.961436,63.564696,51.144055,63.564696,63.923278,40.908053,67.371988,51.144055,43.749019,59.883551,59.883551,51.144055,51.144055,59.883551,70.758537,48.400369,59.883551,63.923278,29.948849,60.656157,58.883022,58.883022,53.961436
83.056714,79.701746,79.701746,79.701746,78.708328,79.701746,83.056714,79.701746,79.701746,68.970893,76.235415,72.173186,76.925105,79.701746,75.775173,77.302788,73.059904,79.701746,72.922495,77.599914,76.235415,78.708328,69.387095,79.701746,66.365884
68.525723,68.55951,52.293216,68.96628,64.513124,57.15198,52.476223,51.552388,60.199392,63.37703,59.586922,77.781669,78.673223,57.732665,53.577749,65.497774,66.285226,67.121458,71.628601,65.909838,57.978904,66.407444,68.685414,53.493307,54.456515
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
50.795006,59.20706,27.4204,61.666292,56.580796,33.935628,40.406258,45.475147,44.36517,47.149587,44.979352,49.808492,62.855769,54.043487,32.238982,60.918594,44.69948,44.918012,73.456588,45.578971,49.55882,42.26009,48.342223,45.404589,55.958596
68.782369,52.521094,47.050531,60.801883,54.699364,64.207819,42.359365,26.221263,42.873251,43.966269,46.59884,54.675369,55.072362,56.828579,27.810865,39.956265,42.74013,60.094759,52.521094,36.772384,54.17488,45.174075,44.189391,45.313935,63.638045
66.703882,59.499584,59.947612,59.542713,64.764782,58.296997,56.35121,50.855882,56.892635,53.957443,54.672494,50.889808,59.947612,64.706324,60.870496,49.448842,68.881668,62.244807,67.152401,52.21769,64.706324,64.007193,56.017589,51.333146,54.790856


##　補正後データ作成

In [23]:
def empty_like(df: pl.DataFrame) -> pl.DataFrame:
    """dfと同じスキーマの空DataFrameを返す"""
    return pl.DataFrame({c: pl.Series([], dtype=t) for c, t in df.schema.items()})

def read_optional_csv(path: str | None, schema_like: pl.DataFrame | None = None) -> pl.DataFrame:
    """パスがあれば読み込み、なければ空DF（可能ならschema_likeに合わせる）"""
    if path:
        return pl.read_csv(path)
    return empty_like(schema_like) if schema_like is not None else pl.DataFrame()

In [24]:
#今回
df_score = pl.read_csv(path)

# 前回・前々回（なければ df_score と同スキーマの空DF）
df_score_prev1 = read_optional_csv(path_prev1, schema_like=df_score)
df_score_prev2 = read_optional_csv(path_prev2, schema_like=df_score)
df_prevs = [df_score_prev1, df_score_prev2]


In [25]:
#ユーザー情報
df_info = pl.read_csv(path_info)
df_info_all = pl.read_csv(path_info_all)


##年齢を年代に変換(年齢の列がある場合)
df_info = df_info.with_columns(
    pl.when(pl.col("年齢") <= 19).then(pl.lit("10代"))
    .when((pl.col("年齢") >= 20) & (pl.col("年齢") <= 29)).then(pl.lit("20代"))
    .when((pl.col("年齢") >= 30) & (pl.col("年齢") <= 39)).then(pl.lit("30代"))
    .when((pl.col("年齢") >= 40) & (pl.col("年齢") <= 49)).then(pl.lit("40代"))
    .when((pl.col("年齢") >= 50) & (pl.col("年齢") <= 59)).then(pl.lit("50代"))
    .otherwise(pl.lit("60代以上"))
    .alias("年代")
)

#使用する属性情報(必要に応じてマージ、リネームしてください)
property_info_cols = [
    "職種", "役職", "所属部門", "性別", "年代"
]

# 欠けている列を追加（"ー" で埋める）
## 必要な列を確認
existing_cols = df_info.columns
missing_cols = [col for col in property_info_cols if col not in existing_cols]
if missing_cols:
    df_info = df_info.with_columns(
        [pl.lit("ー").alias(col) for col in missing_cols]
    )
    

# 最終的に指定順に整列
df_info = df_info.select(["GROW ID", "社員番号", "氏名"] + property_info_cols)

df_info_all = df_info_all.select(["GROW ID", "社員番号", "氏名"])

In [26]:
#スコアデータとマージ
df_score = df_score.join(df_info[["GROW ID", "社員番号", "氏名", "職種", "役職", "所属部門", "性別", "年代"]], on=["GROW ID"], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/3133677257.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score = df_score.join(df_info[["GROW ID", "社員番号", "氏名", "職種", "役職", "所属部門", "性別", "年代"]], on=["GROW ID"], how="left")


In [27]:
def add_mean_columns(df: pl.DataFrame, include_self: bool = True) -> pl.DataFrame:
    """各領域の平均列を追加。空DFでも列は追加"""
    # 空でも列を追加するため、mean_horizontal の結果がなくても with_columns を呼ぶ
    out = df.with_columns(
        pl.lit(None).alias("認知領域平均"),
        pl.lit(None).alias("自己領域平均"),
        pl.lit(None).alias("他者領域平均"),
        pl.lit(None).alias("コミュニティー領域平均"),
        pl.lit(None).alias("全コンピテンシー平均"),
    )

    if not df.is_empty():
        out = out.with_columns(
            pl.mean_horizontal(cog).alias("認知領域平均"),
            pl.mean_horizontal(sel).alias("自己領域平均"),
            pl.mean_horizontal(oth).alias("他者領域平均"),
            pl.mean_horizontal(com).alias("コミュニティー領域平均"),
            pl.mean_horizontal(cn).alias("全コンピテンシー平均"),
        )

    if include_self:
        out = out.with_columns(
            pl.lit(None).alias(f"認知領域平均{self_suffix}"),
            pl.lit(None).alias(f"自己領域平均{self_suffix}"),
            pl.lit(None).alias(f"他者領域平均{self_suffix}"),
            pl.lit(None).alias(f"コミュニティー領域平均{self_suffix}"),
            pl.lit(None).alias(f"全コンピテンシー平均{self_suffix}"),
        )
        if not df.is_empty():
            out = out.with_columns(
                pl.mean_horizontal(cog_self).alias(f"認知領域平均{self_suffix}"),
                pl.mean_horizontal(sel_self).alias(f"自己領域平均{self_suffix}"),
                pl.mean_horizontal(oth_self).alias(f"他者領域平均{self_suffix}"),
                pl.mean_horizontal(com_self).alias(f"コミュニティー領域平均{self_suffix}"),
                pl.mean_horizontal(cn_self).alias(f"全コンピテンシー平均{self_suffix}"),
            )

    return out


# 適用
# 平均列付与（空DFはスキップ）
df_score = add_mean_columns(df_score)
df_prevs = [add_mean_columns(dft) for dft in [df_score_prev1, df_score_prev2]]
benchmark = add_mean_columns(benchmark, include_self=False)

## 評価者傾向

- 評価者傾向を表示しないは無視してください

In [28]:
# 素点インポート
# 共通dtype
common_dtypes = {
    "EVALUATEE_ID": pl.Utf8,
    "EVALUATOR_ID": pl.Utf8,
    "COMPETENCY_NAME": pl.Utf8,
    "SCORE": pl.Float64,
    "ANSWER_TIME": pl.Float64,
    "RELATION_NAME": pl.Utf8,
}

# 今回データ
soten = (
    pl.read_csv(soten_path, dtypes=common_dtypes)
    .select(common_dtypes.keys())
    .with_columns(pl.lit("今回").alias("VERSION"))
)

if prev_soten_paths: 
    # 過去データ
    soten_prevs = []
    for i, path in enumerate(prev_soten_paths, start=1):
        soten_prev = (
            pl.read_csv(path, dtypes=common_dtypes)
            .select(common_dtypes.keys())
            .with_columns(pl.lit(f"前回{i}").alias("VERSION"))
        )
        soten_prevs.append(soten_prev)
    
    # 結合
    soten_all = pl.concat([soten] + soten_prevs)
    
    # 「測定回付きID」を付与（受検者・評価者の両方）
    soten_all = soten_all.with_columns(
        EVALUATEE_ID_by_eval = pl.col("EVALUATEE_ID") + "_" + pl.col("VERSION").cast(str)
    )

else:
   soten_all = soten.clone() 

soten_all.head()

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/1969159092.py:14: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv(soten_path, dtypes=common_dtypes)
/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/1969159092.py:24: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv(path, dtypes=common_dtypes)


EVALUATEE_ID,EVALUATOR_ID,COMPETENCY_NAME,SCORE,ANSWER_TIME,RELATION_NAME,VERSION,EVALUATEE_ID_by_eval
str,str,str,f64,f64,str,str,str
"""F10000011""","""F10000042""","""成長""",4.0,10.189,"""同僚""","""今回""","""F10000011_今回"""
"""F10000031""","""F10000033""","""個人的実行力""",5.0,9.143,"""同僚""","""今回""","""F10000031_今回"""
"""F10000020""","""F10000033""","""個人的実行力""",5.0,9.143,"""同僚""","""今回""","""F10000020_今回"""
"""F10000006""","""F10000045""","""興味""",3.0,6.74,"""同僚""","""今回""","""F10000006_今回"""
"""F10000015""","""F10000045""","""興味""",4.0,6.74,"""部下""","""今回""","""F10000015_今回"""


In [29]:
# ##全受験の評価傾向
evaluator_df_all = soten_all.group_by("EVALUATOR_ID").agg(
    pl.col("SCORE").mean().alias("評価平均点"),#全受験での評価平均点
    # pl.col("SCORE").std().alias("評価標準偏差"),#全受験での評価の標準偏差
    pl.col("EVALUATEE_ID_by_eval").n_unique().alias("累計評価人数"),
)

##今回の受験の評価傾向
evaluator_df = soten.group_by("EVALUATOR_ID").agg(
    pl.col("SCORE").mean().alias("評価平均点"),#今回の受験での評価平均点
    pl.col("EVALUATEE_ID").n_unique().alias("評価人数"),#今回の受験での評価人数
    pl.col("SCORE").std().alias("評価標準偏差"),#今回の受験での評価の標準偏差
    pl.col("ANSWER_TIME").mean().alias("平均回答時間"),
).sort("評価標準偏差")

evaluator_df = evaluator_df.with_columns(
    (pl.col("平均回答時間")/pl.col("評価人数")).alias("一人当たり平均回答時間")
)

In [30]:
# #氏名を付与
evaluator_df = evaluator_df.join(df_info_all.select(
     ["氏名", "GROW ID"]
), left_on = "EVALUATOR_ID", right_on="GROW ID", how="left")

evaluator_df = evaluator_df.rename({
            "氏名": "評価者名"
        })

evaluator_df_all = evaluator_df_all.join(df_info_all.select(
     ["氏名", "GROW ID"]
), left_on = "EVALUATOR_ID", right_on="GROW ID", how="left")

evaluator_df_all = evaluator_df_all.rename({
            "氏名": "評価者名"
        })

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/2259309822.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  evaluator_df = evaluator_df.join(df_info_all.select(
/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/2259309822.py:10: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  evaluator_df_all = evaluator_df_all.join(df_info_all.select(


In [31]:
# # === 今回の測定データを基準にした評価者傾向の分類 ===(素点集計.ipynbで集計)
evaluator_df = evaluator_df.with_columns(
    #評価傾向
    pl.when(pl.col("評価平均点")>4.853333) # 全体上位5%より高い 
    .then(pl.lit("評価甘め傾向"))
    .when(pl.col("評価平均点")>4.313725) # 全体上位25%より高い
    .then(pl.lit("評価やや甘め傾向"))
    .when(pl.col("評価平均点")<2.921569) # 全体下位5%より低い
    .then(pl.lit("評価厳しめ傾向"))
    .when(pl.col("評価平均点")<3.488889) # 全体下位25%より低い
    .then(pl.lit("評価やや厳しめ傾向"))  
    .otherwise(pl.lit("ー")).alias("評価傾向"),

    #評価ばらつき
    pl.when(pl.col("評価標準偏差")<0.310768) # 全体下位5%より低い
    .then(pl.lit("評価単調傾向"))
    .otherwise(pl.lit("ー")).alias("評点ばらつき傾向"),

    pl.when(pl.col("一人当たり平均回答時間")<2.910552) # 全体下位5%より低い
    .then(pl.lit("即答傾向"))
    .otherwise(pl.lit("ー")).alias("回答スピード傾向"),
    
)

In [32]:
# # # === 全素点データを基準にした評価者傾向の分類 ===(素点集計.ipynbで集計)
evaluator_df_all = evaluator_df_all.with_columns(
#     #評価傾向
    pl.when(pl.col("評価平均点")>4.853333) # 全体上位5%より高い 
    .then(pl.lit("評価甘め傾向"))
    .when(pl.col("評価平均点")>4.313725) # 全体上位25%より高い
    .then(pl.lit("評価やや甘め傾向"))
    .when(pl.col("評価平均点")<2.921569) # 全体下位5%より低い
    .then(pl.lit("評価厳しめ傾向"))
    .when(pl.col("評価平均点")<3.488889) # 全体下位25%より低い
    .then(pl.lit("評価やや厳しめ傾向"))  
    .otherwise(pl.lit("ー")).alias("評価傾向(全受験)"),
#     #評価ばらつき
#     pl.when(pl.col("評価標準偏差")<0.310768) # 全体下位5%より低い
#     .then(pl.lit("評価単調傾向"))
#     .otherwise(pl.lit("ー")).alias("評点ばらつき傾向"),
)

In [33]:
# # 評価者と被評価者をマッピング
mapping_df = soten["EVALUATEE_ID", "EVALUATOR_ID", "RELATION_NAME"].unique()
mapping_df = mapping_df.rename({
    "EVALUATEE_ID": "GROW ID"}
)

## クラスタリング(コンピテンシータイプ分類)

growの過去データから作成したコンピテンシータイプモデルで人材分類する

| タイプ名 |     型名    | 概要                                        | 重視軸    | 思考スタイル  | 高めの傾向にあるコンピテンシー                |
| :--: | :-------: | :---------------------------------------- | :----- | :------ | :----------------------------- |
|   ①  |  **突破型**  | 理想や目標に向かって主体的に行動していくタイプ                   | タスク重視  | 未来志向タイプ | 課題設定、創造性、解決意向、ビジョン、影響力の行使、自己効力 |
|   ②  |  **構築型**  | 論理的かつ冷静に筋道立てて成果を積み上げるタイプ                  | タスク重視  | 現実志向タイプ | 論理的思考、疑う力、解決意向、感情コントロール、自己効力   |
|   ③  |  **支援型**  | 周囲との調和を重んじ、現実的かつ落ち着いた姿勢でチームを支えるタイプ        | 人間関係重視 | 現実志向タイプ | 寛容、共感傾聴力、感情コントロール、外交性、地球市民     |
|   ④  | **巻き込み型** | 人とのつながりを大切にしながら、共感と信頼を基盤に新しい価値や変化を生み出すタイプ | 人間関係重視 | 未来志向タイプ | 外交性、共感傾聴力、成長、興味、地球市民           |


In [34]:
## モデルロード
#pca ロード
pca = joblib.load(pca_path)
# 保存したUMAPモデルを読み込む
umap = joblib.load(umap_path)
# 保存したkmeansモデルを読み込む
kmeans = joblib.load(kmeans_path)

In [35]:
def assign_competency_types(
    df_score: pl.DataFrame,
    cn: list[str],
    pca,
    umap,
    kmeans,
    suffix: str = "",
    col_prefix: str = "",
    cluster_colname: str = "cluster"
) -> pl.DataFrame:
    """
    スコアデータに対して、標準化・PCA・UMAP・クラスタリングを行い、
    重視軸、スタンス軸、タイプ分類を追加する関数。
    """

    # カラムリネームが必要な場合
    if col_prefix:
        df_score = df_score.rename({f"{col_prefix}{col}" : col for col in cn})

    # null行除去
    df_score_dn = df_score.drop_nulls(subset=cn)

    # -------------------------
    # 0行の場合：列だけ追加して返す
    # -------------------------
    if df_score_dn.height == 0:
        return df_score.with_columns(
            pl.lit(None).cast(pl.Int64).alias(cluster_colname),
            pl.mean_horizontal(empathy_stability).alias(f"協調・共感平均{suffix}"),
            pl.mean_horizontal(problem_solving_active).alias(f"課題志向平均{suffix}"),
            pl.mean_horizontal(passion_driver).alias(f"未来志向平均{suffix}"),
            pl.mean_horizontal(logic_reflection).alias(f"現実志向平均{suffix}"),
            pl.lit(None).cast(pl.Utf8).alias(f"重視軸{suffix}"),
            pl.lit(None).cast(pl.Utf8).alias(f"スタンス軸{suffix}"),
            pl.lit(None).cast(pl.Utf8).alias(f"コンピテンシータイプ{suffix}"),
        )

    # ここから先は 1行以上ある前提で通常処理
    X = df_score_dn.select(cn).to_numpy()

    # 行ごとに標準化
    row_mean = X.mean(axis=1, keepdims=True)
    row_std = X.std(axis=1, ddof=0, keepdims=True)
    # 0分散対策：std=0 は 1 に置換
    zero_var = (row_std == 0)
    row_std[zero_var] = 1.0
    X_std = (X - row_mean) / row_std

    # PCA・UMAP・クラスタリング
    pca_row = pca.transform(X_std)
    X_reduced = pl.DataFrame(data=pca_row, schema=[f"PC{i+1}" for i in range(pca_row.shape[1])])
    Y_score = umap.transform(X_reduced[:, :2])
    cluster_score = kmeans.predict(Y_score)

    # クラスタ列追加
    df_score_dn = df_score_dn.with_columns(
        pl.Series(cluster_colname, cluster_score)
    )

    # タイプごとのコンピテンシー定義
    problem_solving_active = ["課題設定", "論理的思考", "解決意向", "創造性", "疑う力", "影響力の行使", "自己効力", "内的価値"]
    empathy_stability = ["寛容", "感情コントロール", "耐性", "外交性", "組織へのコミットメント", "共感・傾聴力", "地球市民"]
    passion_driver = ["外交性", "創造性", "ヴィジョン", '興味',  '地球市民',  '成長']
    logic_reflection = ['解決意向', '論理的思考', '疑う力','感情コントロール', '共感・傾聴力', '寛容']

    # 平均スコア算出
    df_score_dn = df_score_dn.with_columns(
        pl.mean_horizontal(empathy_stability).alias(f"協調・共感平均{suffix}"),
        pl.mean_horizontal(problem_solving_active).alias(f"課題志向平均{suffix}"),
        pl.mean_horizontal(passion_driver).alias(f"未来志向平均{suffix}"),
        pl.mean_horizontal(logic_reflection).alias(f"現実志向平均{suffix}"),
    )

    # 重視軸とスタンス軸の分類
    df_score_dn = df_score_dn.with_columns(
        pl.when(pl.col(cluster_colname).is_in([1, 3]))
        .then(pl.lit("タスク重視タイプ"))
        .otherwise(pl.lit("人間関係重視タイプ"))
        .alias(f"重視軸{suffix}"),

        pl.when(pl.col(cluster_colname).is_in([2, 3]))
        .then(pl.lit("現実志向タイプ"))
        .otherwise(pl.lit("未来志向タイプ"))
        .alias(f"スタンス軸{suffix}"),
    )

    # コンピテンシータイプの最終分類
    df_score_dn = df_score_dn.with_columns(
        pl.when((pl.col(f"重視軸{suffix}") == "タスク重視タイプ") & (pl.col(f"スタンス軸{suffix}") == "未来志向タイプ"))
        .then(pl.lit("突破型"))
        .when((pl.col(f"重視軸{suffix}") == "人間関係重視タイプ") & (pl.col(f"スタンス軸{suffix}") == "未来志向タイプ"))
        .then(pl.lit("巻き込み型"))
        .when((pl.col(f"重視軸{suffix}") == "タスク重視タイプ") & (pl.col(f"スタンス軸{suffix}") == "現実志向タイプ"))
        .then(pl.lit("構築型"))
        .otherwise(pl.lit("支援型"))
        .alias(f"コンピテンシータイプ{suffix}")
    )

    return df_score_dn



In [36]:
# 他者評価データ
df_score = assign_competency_types(
    df_score=df_score,
    cn=cn,
    pca=pca,
    umap=umap,
    kmeans=kmeans,
    suffix="",
    cluster_colname="cluster"
)

# 自己評価データ
df_score_self_base = df_score.select(
    ["GROW ID"] + [f"{col}{self_suffix}" for col in cn]
)
df_score_self_renamed = df_score_self_base.rename({
    f"{col}{self_suffix}": col for col in cn
})

df_score_self_processed = assign_competency_types(
    df_score=df_score_self_renamed,
    cn=cn,
    pca=pca,
    umap=umap,
    kmeans=kmeans,
    suffix="(自己)",
    col_prefix="",
    cluster_colname="cluster_self"
)

In [37]:
df_score = df_score.join(df_score_self_processed.select(["GROW ID", "重視軸(自己)", "スタンス軸(自己)", "コンピテンシータイプ(自己)"]), on="GROW ID")

In [38]:
## 確認用
# タイプ①：先導タイプ
problem_solving_active = [
    "課題設定",
    "論理的思考",
    "解決意向",
    "創造性",
    "疑う力",
    "影響力の行使",
    "自己効力",
    "内的価値"
]

# タイプ②：協調タイプ
empathy_stability = [
    "寛容",
    "感情コントロール",
    "耐性",
    "外交性",
    "組織へのコミットメント",
    "共感・傾聴力",
    "地球市民"
]

# タイプ③：未来志向タイプ
passion_driver = [
    "外交性",
    "創造性",
    "ヴィジョン",
    '興味', 
    '地球市民', 
    '成長'
]

# タイプ④：現実志向タイプ
logic_reflection = [
    '解決意向', '論理的思考', '疑う力','感情コントロール', '共感・傾聴力', '寛容'
]

dft = df_score.clone()

dft = dft.with_columns(
     pl.mean_horizontal(empathy_stability).alias("協調平均"),
    pl.mean_horizontal(problem_solving_active).alias("先導平均"),
    pl.mean_horizontal(passion_driver).alias("未来志向平均"),
    pl.mean_horizontal(logic_reflection).alias("現実志向平均"),
)

dft.group_by(
    ["コンピテンシータイプ", "重視軸", "スタンス軸"]
).agg(
   pl.mean(["協調平均", "先導平均", "未来志向平均", "現実志向平均"])
)

コンピテンシータイプ,重視軸,スタンス軸,協調平均,先導平均,未来志向平均,現実志向平均
str,str,str,f64,f64,f64,f64
"""突破型""","""タスク重視タイプ""","""未来志向タイプ""",60.179835,63.282303,56.805578,63.201188
"""構築型""","""タスク重視タイプ""","""現実志向タイプ""",74.638894,77.953017,64.861306,80.3308
"""支援型""","""人間関係重視タイプ""","""現実志向タイプ""",64.602553,55.156121,51.0406,65.033882
"""巻き込み型""","""人間関係重視タイプ""","""未来志向タイプ""",69.777194,60.598881,62.24245,65.830281


## 評価者情報マージ

#### 評価者情報を載せない場合

In [39]:
cols = [
    f"{col}_rank_{i}"
    for i in range(1, 8)
    for col in ["評価者名","RELATION_NAME", "評価傾向", "評点ばらつき傾向", "回答スピード傾向", '評価人数', '評価傾向(全受験)', "累計評価人数"]
]

df_pivot_evaluator = df_score.select(["GROW ID"]).with_columns(
    [pl.lit("ー").alias(c) for c in cols]
)

In [40]:
#今回受験の評価傾向をマージ
mapping_df = mapping_df.join(evaluator_df.select(
    ['EVALUATOR_ID',"評価者名", '評点ばらつき傾向', "回答スピード傾向", "評価傾向", "評価人数"]), 
    on='EVALUATOR_ID' , how="left"
)
#受験全体の評価傾向をマージ
mapping_df = mapping_df.join(evaluator_df_all.select(
    ['EVALUATOR_ID','評価傾向(全受験)', "累計評価人数"]), 
    on='EVALUATOR_ID' , how="left"
)

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/1180229504.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  mapping_df = mapping_df.join(evaluator_df.select(
/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/1180229504.py:7: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  mapping_df = mapping_df.join(evaluator_df_all.select(


In [41]:
mapping_df = mapping_df.with_columns(
     pl.col('EVALUATOR_ID').rank(method="dense").over("GROW ID").cast(pl.Int64).alias("rank")
)

In [42]:
df_pivot_evaluator = mapping_df.pivot(values=["評価者名","RELATION_NAME", "評価傾向", "評点ばらつき傾向", "回答スピード傾向", '評価人数', '評価傾向(全受験)', "累計評価人数"], index="GROW ID", columns="rank", aggregate_function="first")

In [43]:
# rankの最大値を取得
rank_cols = [c for c in df_pivot_evaluator.columns if "_rank_" in c]
max_rank = max(int(c.split("_")[-1]) for c in rank_cols)

# 10未満なら欠損列を追加
if max_rank < 10:
    for i in range(max_rank + 1, 8):
        for base in ["評価者名","RELATION_NAME", "評価傾向", "評点ばらつき傾向", "回答スピード傾向", '評価人数', '評価傾向(全受験)', "累計評価人数"]:
            df_pivot_evaluator = df_pivot_evaluator.with_columns(pl.lit(None).alias(f"{base}_rank_{i}"))

In [44]:
cols = [c for c in df_pivot_evaluator.columns if c != "GROW ID"]

# 評価者ごとのrank番号を抽出（例: rank1, rank2...）
from collections import defaultdict
ranked_cols = defaultdict(dict)
for col in cols:
    key, rank = col.split("_rank_")
    ranked_cols[int(rank)][key] = col

In [45]:
ordered_cols = ["GROW ID"] + [
    col_name
    for rank in sorted(ranked_cols)
    for col_name in [ranked_cols[rank].get("評価者名"), ranked_cols[rank].get("RELATION_NAME"), ranked_cols[rank].get("評価傾向"), 
                     ranked_cols[rank].get("評点ばらつき傾向"), ranked_cols[rank].get("回答スピード傾向"), ranked_cols[rank].get("評価人数"), 
                     ranked_cols[rank].get("評価傾向(全受験)"), ranked_cols[rank].get("累計評価人数")]
    if col_name is not None
]

In [46]:
df_pivot_evaluator = df_pivot_evaluator.select(ordered_cols).fill_null("ー")

## マッチング

- growのスコアが似ている人(波形)
- 苦手なコンピテンシーが得意な人

を各3名ピックアップ

### 似てる人

In [47]:
# ---------- ① データ準備 ----------
# IDとスコア部分
df_score_comp = df_score.select(["GROW ID", "氏名"] + cn).drop([col for col in cn if df_score[col].is_null().all()])
ids = df_score_comp["GROW ID"].to_list()
names = df_score_comp["氏名"].to_list()
comp_cols = [col for col in df_score_comp.columns if col in cn]
X = df_score_comp.select(comp_cols).to_numpy()

# ---------- ② 横方向にZスコア標準化 ----------
row_mean = X.mean(axis=1, keepdims=True)
row_std = X.std(axis=1, ddof=0, keepdims=True)
X_std = (X - row_mean) / row_std

# ---------- ③ PCA & コサイン類似度計算 ----------
n_components = min(15, X_std.shape[1])  # 特徴量より成分数が多くならないように

try:
    # すでに学習済みの pca を使う
    pca_row = pca.transform(X_std)

except ValueError as e:
    # 特徴量数ミスマッチのときだけ作り直す
    if "features" in str(e):
        # PCA を作り直して学習し直す
        pca = PCA(n_components=n_components)
        pca_row = pca.fit_transform(X_std)
        print("PCA を再学習しました（特徴量数が変わったため）。")
    else:
        # それ以外のエラーはそのまま投げる
        raise

# 以降はそのまま
sim_matrix = cosine_similarity(pca_row[:, :15])

# ---------- ④ 結果整形 ----------
top_k = 3  # 上位3人まで
result_rows = []

for i, l in enumerate(zip(ids, names)):
    id_, name = l
    sims = sim_matrix[i]
    # 自分自身を除く
    sim_series = [(names[j], sims[j]) for j in range(len(ids)) if j != i]
    # 類似度の高い順に並べて上位top_k件を取得
    top_similar = sorted(sim_series, key=lambda x: x[1], reverse=True)[:top_k]

    # 結果行作成
    row = {
        "GROW ID": id_,
        "対象者": name,
    }
    for rank, (match_id, score) in enumerate(top_similar, start=1):
        row[f"似ている人{rank}"] = match_id
        row[f"類似度{rank}"] = round(score, 4)
    result_rows.append(row)

# ---------- ⑤ Polars DataFrameに変換 ----------
similar_df = pl.DataFrame(result_rows)

### 似てない人(使ってない)

In [48]:
top_k = 3  # 下位3人まで
result_rows = []

for i, l in enumerate(zip(ids, names)):
    id_, name = l
    sims = sim_matrix[i]
    # 自分自身を除く
    sim_series = [(names[j], sims[j]) for j in range(len(ids)) if j != i]
    # 類似度が低い順（= 似ていない人）に並べて上位top_k件を取得
    least_similar = sorted(sim_series, key=lambda x: x[1])[:top_k]

    # 結果行作成
    row = {
        "GROW ID": id_,
        "対象者": name,
    }
    for rank, (match_id, score) in enumerate(least_similar, start=1):
        row[f"似ていない人{rank}"] = match_id
        row[f"類似度{rank}"] = round(score, 4)
    result_rows.append(row)

# 出力用 DataFrame
dissimilar_df = pl.DataFrame(result_rows)

### 苦手なコンピテンシーが得意な人

In [49]:
# ----- 前提：df_score, comp_cols の定義 -----
# df_score: GROW ID, 氏名, comp_1 ~ comp_25 がある横持ちデータ

# ----- Step 1: 全コンピテンシーの降順ランクを事前計算 -----
df_rank = df_score.select(["GROW ID", "氏名"] + comp_cols)

for c in comp_cols:
    df_rank = df_rank.with_columns(
        pl.col(c).rank(method="min", descending=True).alias(f"{c}_rank")
    )

# ----- Step 2: 各人に対して苦手項目3つ → 得意な人3人 抽出 -----
results = []

for row in df_score.iter_rows(named=True):
    GROW_ID = row["GROW ID"]
    name = row["氏名"]
    
    # 対象者のスコア辞書
    scores = {k: row[k] for k in comp_cols}
    weak_cols = sorted(scores.items(), key=lambda x: x[1])[:3]
    weak_comp = [k for k, _ in weak_cols]
    weak_rank_cols = [f"{c}_rank" for c in weak_comp]
    
    # 他の人の中で、苦手項目の順位合計が低い人を抽出
    filtered = (
        df_rank
        .filter(pl.col("GROW ID") != GROW_ID)
        .with_columns(
            pl.sum_horizontal([pl.col(c) for c in weak_rank_cols]).alias("rank_sum")
        )
        .sort("rank_sum")
        .select(["GROW ID", "氏名", "rank_sum"])
        .head(3)
    )

    row_result = {
        "GROW ID": GROW_ID,
        "氏名": name,
    }

    for i, p in enumerate(filtered.iter_rows(named=True), start=1):
        row_result[f"得意な人{i}"] = p["氏名"]

    results.append(row_result)

# ----- Step 3: 結果を Polars DataFrame に -----
df_result = pl.DataFrame(results)

In [50]:
df_result

GROW ID,氏名,得意な人1,得意な人2,得意な人3
str,str,str,str,str
"""F10000042""","""sample42""","""sample44""","""sample55""","""sample45"""
"""F10000030""","""sample20""","""sample45""","""sample55""","""sample44"""
"""F10000011""","""sample4""","""sample45""","""sample44""","""sample13"""
"""F10000053""","""sample43""","""sample13""","""sample44""","""sample45"""
"""F10000013""","""sample23""","""sample45""","""sample44""","""sample55"""
…,…,…,…,…
"""F10000015""","""sample37""","""sample44""","""sample45""","""sample13"""
"""F10000045""","""sample14""","""sample44""","""sample45""","""sample55"""
"""F10000032""","""sample52""","""sample13""","""sample44""","""sample45"""


### マージ

In [51]:
# スコア部分をNumPyに変換
X = df_score.select(comp_cols).to_numpy()
ids = df_score["GROW ID"].to_list()
names = df_score["氏名"].to_list()

# ----- 行方向ランクを計算（降順） -----
# 各行に対して、高い順にランクをつける（高得点=1位）
row_ranks = np.argsort(-X, axis=1).argsort(axis=1) + 1  # 1-based rank（高い値=小さいランク）

# ----- スコアデータに戻す -----
# 各行で苦手な3項目（ランクが高い=値が小さい）を取得
results = []

for i in range(len(X)):
    id_ = ids[i]
    name = names[i]
    row = X[i]
    rank_row = row_ranks[i]
    
    # ランクが高い順にソート（25位,24位,23位...）→苦手項目3つ
    sorted_idx = np.argsort(rank_row)[-3:]  # 下位3項目のindex
    sorted_idx_good = np.argsort(rank_row)[:3]  # 下位3項目のindex
    
    weak_comp = [comp_cols[j] for j in sorted_idx]
    good_comp = [comp_cols[j] for j in sorted_idx_good]

    # 他人の中で、この3項目のスコアが高い人を探す
    X_others = np.delete(row_ranks, i, axis=0)
    names_others = [n for j, n in enumerate(names) if j != i]

    # この3項目のスコア合計が高い人を上位3名抽出
    sums = X_others[:, sorted_idx].sum(axis=1)
    top_idx = np.argsort(sums)[:3]
    top_people = [names_others[j] for j in top_idx]
    row_result = {
        "GROW ID": id_,
        "氏名": name,
    }
    for k, p in enumerate(weak_comp, 1):
        row_result[f"苦手コンピテンシー{k}"] = p
    for k, p in enumerate(good_comp, 1):
        row_result[f"得意コンピテンシー{k}"] = p
    for k, p in enumerate(top_people, 1):
        row_result[f"得意な人{k}"] = p
    results.append(row_result)

# Polarsに戻す
df_weak_good = pl.DataFrame(results)

### 自己評価

In [52]:
## 苦手得意(自己認識)
# スコア部分をNumPyに変換
comp_cols_self = [f"{val}{self_suffix}" for val in comp_cols]
X = df_score.select(comp_cols_self).to_numpy()
ids = df_score["GROW ID"].to_list()
names = df_score["氏名"].to_list()

# ----- 行方向ランクを計算（降順） -----
# 各行に対して、高い順にランクをつける（高得点=1位）
row_ranks = np.argsort(-X, axis=1).argsort(axis=1) + 1  # 1-based rank（高い値=小さいランク）

# ----- スコアデータに戻す -----
# 各行で苦手な3項目（ランクが高い=値が小さい）を取得
results = []

for i in range(len(X)):
    id_ = ids[i]
    name = names[i]
    row = X[i]
    rank_row = row_ranks[i]
    
    # ランクが高い順にソート（25位,24位,23位...）→苦手項目3つ
    sorted_idx = np.argsort(rank_row)[-3:]  # 下位3項目のindex
    sorted_idx_good = np.argsort(rank_row)[:3]  # 下位3項目のindex
    
    weak_comp = [comp_cols[j] for j in sorted_idx]
    good_comp = [comp_cols[j] for j in sorted_idx_good]
    row_result = {
        "GROW ID": id_,
        "氏名": name,
    }
    for k, p in enumerate(weak_comp, 1):
        row_result[f"苦手コンピテンシー(自己評価){k}"] = p
    for k, p in enumerate(good_comp, 1):
        row_result[f"得意コンピテンシー(自己評価){k}"] = p
    results.append(row_result)

# Polarsに戻す
df_weak_good_self = pl.DataFrame(results)

In [53]:
# スコア部分をNumPyに変換
X = df_score.select(comp_cols).to_numpy()
ids = df_score["GROW ID"].to_list()
names = df_score["氏名"].to_list()

# ----- 行方向ランクを計算（降順） -----
# 各人について、値が高いほど低いランク（1が最大）
row_ranks = np.argsort(-X, axis=1).argsort(axis=1) + 1  # 1-based

# ランクデータの列名
rank_colnames = [f"{c}_rank" for c in comp_cols]

# ランクを Polars DataFrame に変換
df_score_ranked = pl.DataFrame(row_ranks, schema=rank_colnames)
df_score_ranked = df_score.select(["GROW ID", "氏名"]).hstack(df_score_ranked)

In [54]:
df_weak_good

GROW ID,氏名,苦手コンピテンシー1,苦手コンピテンシー2,苦手コンピテンシー3,得意コンピテンシー1,得意コンピテンシー2,得意コンピテンシー3,得意な人1,得意な人2,得意な人3
str,str,str,str,str,str,str,str,str,str,str
"""F10000042""","""sample42""","""ヴィジョン""","""組織へのコミットメント""","""地球市民""","""寛容""","""耐性""","""論理的思考""","""sample44""","""sample41""","""sample20"""
"""F10000030""","""sample20""","""課題設定""","""表現力""","""疑う力""","""外交性""","""寛容""","""誠実さ""","""sample45""","""sample2""","""sample47"""
"""F10000011""","""sample4""","""表現力""","""影響力の行使""","""創造性""","""耐性""","""疑う力""","""寛容""","""sample22""","""sample32""","""sample33"""
"""F10000053""","""sample43""","""創造性""","""ヴィジョン""","""影響力の行使""","""寛容""","""感情コントロール""","""耐性""","""sample32""","""sample22""","""sample44"""
"""F10000013""","""sample23""","""影響力の行使""","""共感・傾聴力""","""表現力""","""感情コントロール""","""耐性""","""論理的思考""","""sample33""","""sample8""","""sample42"""
…,…,…,…,…,…,…,…,…,…,…
"""F10000015""","""sample37""","""影響力の行使""","""地球市民""","""創造性""","""誠実さ""","""決断力""","""個人的実行力""","""sample22""","""sample20""","""sample48"""
"""F10000045""","""sample14""","""創造性""","""疑う力""","""地球市民""","""決断力""","""誠実さ""","""情熱・宣教力""","""sample32""","""sample12""","""sample22"""
"""F10000032""","""sample52""","""創造性""","""ヴィジョン""","""影響力の行使""","""論理的思考""","""寛容""","""感情コントロール""","""sample32""","""sample22""","""sample44"""


## 縦持ちデータに変換

In [56]:
#縦持ち変換
df_score_long = df_score.melt(
    id_vars="GROW ID", value_vars=all_items, variable_name="コンピテンシー", value_name="スコア"
).sort("GROW ID")



In [57]:
# コンピランク
def score_to_grade(score):
    if score is None:
        return None
    elif score >= 85:
        return "S"
    elif score >= 70:
        return "A"
    elif score >= 55:
        return "B"
    elif score >= 40:
        return "C"
    else:
        return "D"

# DataFrameが df_comp だとする（GROW ID, コンピテンシー, スコア）

df_score_long = df_score_long.with_columns(
    pl.col("スコア").map_elements(score_to_grade, return_dtype=str).alias("評価")
)
df_score_long = df_score_long.with_columns(
    pl.col("スコア").map_elements(score_to_grade, return_dtype=str).alias("評価")
)

In [58]:
#自己評価
df_score_long_self = df_score.melt(
    id_vars="GROW ID", value_vars=all_items_self, variable_name="コンピテンシー", value_name="スコア(自己評価)"
)

df_score_long_self = df_score_long_self.with_columns(
    pl.col("コンピテンシー").str.replace(f"{self_suffix}", "").alias("コンピテンシー")
)
#マージ
df_score_long = df_score_long.join(df_score_long_self, on=["GROW ID", "コンピテンシー"])

In [60]:
#前回の結果をマージ
for i, dft in enumerate(df_prevs, start=1):
    

    df_score_long_prev = dft.melt(
        id_vars="GROW ID", value_vars=all_items, variable_name="コンピテンシー", value_name=f"スコア_prev{i}"
    ).sort("GROW ID")
    
    #自己評価
    df_score_long_self_prev = dft.melt(
        id_vars="GROW ID", value_vars=all_items_self, variable_name="コンピテンシー", value_name=f"スコア(自己評価)_prev{i}"
    )
    
    df_score_long_self_prev =  df_score_long_self_prev.with_columns(
        pl.col("コンピテンシー").str.replace(f"{self_suffix}", "").alias("コンピテンシー")
    )
    
    #マージ
    df_score_long = df_score_long.join(df_score_long_prev, on=["GROW ID", "コンピテンシー"], how="left")
    df_score_long = df_score_long.join(df_score_long_self_prev, on=["GROW ID", "コンピテンシー"], how="left")


/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/3176326794.py:19: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(df_score_long_prev, on=["GROW ID", "コンピテンシー"], how="left")
/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/3176326794.py:20: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(df_score_long_self_prev, on=["GROW ID", "コンピテンシー"], how="left")


In [64]:
#ジョハリ
def classify_johari_item(peer, self):
     # null / NaN を先に処理
    if peer is None or self is None:
        return None
    if isinstance(peer, float) and math.isnan(peer):
        return None
    if isinstance(self, float) and math.isnan(self):
        return None
        
    total = peer + self
    diff = abs(peer - self)
    if (total >= 165) or (total >= 130 and diff <= 10):
        return "開放の窓"
    elif (peer >= 60) and (peer >= self):
        return "盲点の窓"
    elif (self >= 60) and (self > peer):
        return "秘密の窓"
    else:
        return "未知の窓"

df_score_long = df_score_long.with_columns([
        pl.struct(["スコア", "スコア(自己評価)"]).map_elements(
            lambda d: classify_johari_item(d["スコア"], d["スコア(自己評価)"]), 
            return_dtype=str
        ).alias(f"ジョハリ")
    ])

In [65]:
df_wide = df_score.select(
    ['GROW ID',
 '氏名',
 "職種", "役職", "所属部門", "性別", "年代",
 '重視軸',
 'スタンス軸',
 'コンピテンシータイプ',
 '重視軸(自己)',
 'スタンス軸(自己)',
 'コンピテンシータイプ(自己)',
 # 'メタ認知度'
    ]
)

df_score_long = df_wide.join(df_score_long, on=['GROW ID',], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/644723968.py:15: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_wide.join(df_score_long, on=['GROW ID',], how="left")


In [66]:
#評価者情報マージ
df_score_long = df_score_long.join(df_pivot_evaluator, on=['GROW ID',], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/4143548336.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(df_pivot_evaluator, on=['GROW ID',], how="left")


In [67]:
# 似ている人情報
df_score_long = df_score_long.join(similar_df.select(
    ['GROW ID','似ている人1', '類似度1', '似ている人2', '類似度2', '似ている人3', '類似度3']
), on=['GROW ID',], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/2505451218.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(similar_df.select(


In [68]:
# 苦手を補完できる人情報
df_score_long = df_score_long.join(df_weak_good.select(
    ['GROW ID',
 '苦手コンピテンシー1',
 '苦手コンピテンシー2',
 '苦手コンピテンシー3',
 '得意コンピテンシー1',
 '得意コンピテンシー2',
 '得意コンピテンシー3',
 '得意な人1',
 '得意な人2',
 '得意な人3']
), on=['GROW ID',], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/389271636.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(df_weak_good.select(


In [69]:
# 苦手得意(自己評価)
df_score_long = df_score_long.join(df_weak_good_self.select(
    ['GROW ID',
 '苦手コンピテンシー(自己評価)1',
 '苦手コンピテンシー(自己評価)2',
 '苦手コンピテンシー(自己評価)3',
 '得意コンピテンシー(自己評価)1',
 '得意コンピテンシー(自己評価)2',
 '得意コンピテンシー(自己評価)3',
]
), on=['GROW ID',], how="left")

/var/folders/pw/xvty5dzj37d98jlgrnmd6bb40000gp/T/ipykernel_74397/2276966502.py:2: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df_score_long = df_score_long.join(df_weak_good_self.select(


In [70]:
df_score_long = df_score_long.with_columns(
    自己他者乖離 = pl.col("スコア") - pl.col("スコア(自己評価)"),
    前回比較 = pl.col("スコア") - pl.col("スコア_prev1"),
)

In [71]:
# idごとにrow_countをつけて、最初の行を判定
df_score_long = df_score_long.sort("GROW ID").with_columns(
    (pl.col("GROW ID") != pl.col("GROW ID").shift(1)).cast(pl.Int8).fill_null(1).alias("最初フラグ")
)

In [72]:
df_score_long.columns

['GROW ID',
 '氏名',
 '職種',
 '役職',
 '所属部門',
 '性別',
 '年代',
 '重視軸',
 'スタンス軸',
 'コンピテンシータイプ',
 '重視軸(自己)',
 'スタンス軸(自己)',
 'コンピテンシータイプ(自己)',
 '氏名_right',
 '職種_right',
 '役職_right',
 '所属部門_right',
 '性別_right',
 '年代_right',
 '重視軸_right',
 'スタンス軸_right',
 'コンピテンシータイプ_right',
 '重視軸(自己)_right',
 'スタンス軸(自己)_right',
 'コンピテンシータイプ(自己)_right',
 'コンピテンシー',
 'スコア',
 '評価',
 'スコア(自己評価)',
 'スコア_prev1',
 'スコア(自己評価)_prev1',
 'スコア_prev2',
 'スコア(自己評価)_prev2',
 'ジョハリ',
 '評価者名_rank_1',
 'RELATION_NAME_rank_1',
 '評価傾向_rank_1',
 '評点ばらつき傾向_rank_1',
 '回答スピード傾向_rank_1',
 '評価人数_rank_1',
 '評価傾向(全受験)_rank_1',
 '累計評価人数_rank_1',
 '評価者名_rank_2',
 'RELATION_NAME_rank_2',
 '評価傾向_rank_2',
 '評点ばらつき傾向_rank_2',
 '回答スピード傾向_rank_2',
 '評価人数_rank_2',
 '評価傾向(全受験)_rank_2',
 '累計評価人数_rank_2',
 '評価者名_rank_3',
 'RELATION_NAME_rank_3',
 '評価傾向_rank_3',
 '評点ばらつき傾向_rank_3',
 '回答スピード傾向_rank_3',
 '評価人数_rank_3',
 '評価傾向(全受験)_rank_3',
 '累計評価人数_rank_3',
 '評価者名_rank_4',
 'RELATION_NAME_rank_4',
 '評価傾向_rank_4',
 '評点ばらつき傾向_rank_4',
 '回答スピード傾向_rank_4',
 

## ベンチマークサマリー

In [73]:
df_summary = pl.DataFrame({
    "項目": all_items,
    "平均": [benchmark[col].mean() for col in all_items],
    "中央値": [benchmark[col].median() for col in all_items],
    "標準偏差": [benchmark[col].std() for col in all_items],
    "q5": [benchmark[col].quantile(0.05, "nearest") for col in all_items],
    "q10": [benchmark[col].quantile(0.10, "nearest") for col in all_items],
    "q20": [benchmark[col].quantile(0.20, "nearest") for col in all_items],
    "q25": [benchmark[col].quantile(0.25, "nearest") for col in all_items],    
    "q75": [benchmark[col].quantile(0.75, "nearest") for col in all_items],
    "q80": [benchmark[col].quantile(0.80, "nearest") for col in all_items],
    "q90": [benchmark[col].quantile(0.90, "nearest") for col in all_items],
    "q95": [benchmark[col].quantile(0.95, "nearest") for col in all_items],
    
})

In [74]:
df_summary = df_summary.with_columns(
    参考範囲 = pl.col("q25").round(1).cast(str) + "~" + pl.col("q75").round(1).cast(str)
)

In [75]:
df_summary

項目,平均,中央値,標準偏差,q5,q10,q20,q25,q75,q80,q90,q95,参考範囲
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""課題設定""",65.023619,66.882433,13.538851,39.860786,46.382733,53.786056,56.498227,75.369677,77.105933,81.054005,83.726969,"""56.5~75.4"""
"""解決意向""",64.564824,66.713284,12.792947,39.489752,46.868933,54.731559,57.453257,73.960081,75.451887,79.011471,81.525255,"""57.5~74.0"""
"""創造性""",54.28587,55.015388,12.625785,31.893938,37.290392,43.860876,46.203261,63.206737,65.111566,70.005245,73.92371,"""46.2~63.2"""
"""論理的思考""",63.552515,65.426633,13.149809,38.383508,45.450814,53.055444,55.697447,73.364594,75.029098,78.910508,81.583277,"""55.7~73.4"""
"""疑う力""",64.064592,66.076905,12.038626,40.925337,47.620437,54.871286,57.398521,72.698657,74.135428,77.695697,80.23348,"""57.4~72.7"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""地球市民""",51.466368,51.823035,7.740365,38.375779,41.997801,45.690138,46.977551,56.187832,57.262343,60.44093,63.506846,"""47.0~56.2"""
"""組織へのコミットメント""",55.548251,56.377575,10.052235,37.734467,42.690973,47.752846,49.554009,62.348636,63.691047,67.339254,70.365845,"""49.6~62.3"""
"""誠実さ""",62.587191,63.483173,8.313782,47.753777,51.938565,56.392461,57.953028,68.119134,69.199926,72.018764,74.415524,"""58.0~68.1"""


## 保存

In [76]:
from xlsxwriter import Workbook
with Workbook("output/" + filename) as wb:  
    # basic/default conditional formatting
    # df_wide.write_excel(
    #     workbook=wb,
    #     worksheet="コンピテンシーデータ横持ち",
    # )
    df_score_long.write_excel(
        workbook=wb,
        worksheet="コンピテンシーデータ縦持ち",
    )
    df_summary.write_excel(
        workbook=wb,
        worksheet="ベンチマーク集約値",
    ),
    df_info.filter(
            pl.col("GROW ID").is_in(df_score_long["GROW ID"].unique())
        ).write_excel(
        workbook=wb,
        worksheet="受験者情報",
    ),
  
   

In [77]:
df_score_long

GROW ID,氏名,職種,役職,所属部門,性別,年代,重視軸,スタンス軸,コンピテンシータイプ,重視軸(自己),スタンス軸(自己),コンピテンシータイプ(自己),氏名_right,職種_right,役職_right,所属部門_right,性別_right,年代_right,重視軸_right,スタンス軸_right,コンピテンシータイプ_right,重視軸(自己)_right,スタンス軸(自己)_right,コンピテンシータイプ(自己)_right,コンピテンシー,スコア,評価,スコア(自己評価),スコア_prev1,スコア(自己評価)_prev1,スコア_prev2,スコア(自己評価)_prev2,ジョハリ,評価者名_rank_1,RELATION_NAME_rank_1,評価傾向_rank_1,…,評点ばらつき傾向_rank_6,回答スピード傾向_rank_6,評価人数_rank_6,評価傾向(全受験)_rank_6,累計評価人数_rank_6,評価者名_rank_7,RELATION_NAME_rank_7,評価傾向_rank_7,評点ばらつき傾向_rank_7,回答スピード傾向_rank_7,評価人数_rank_7,評価傾向(全受験)_rank_7,累計評価人数_rank_7,似ている人1,類似度1,似ている人2,類似度2,似ている人3,類似度3,苦手コンピテンシー1,苦手コンピテンシー2,苦手コンピテンシー3,得意コンピテンシー1,得意コンピテンシー2,得意コンピテンシー3,得意な人1,得意な人2,得意な人3,苦手コンピテンシー(自己評価)1,苦手コンピテンシー(自己評価)2,苦手コンピテンシー(自己評価)3,得意コンピテンシー(自己評価)1,得意コンピテンシー(自己評価)2,得意コンピテンシー(自己評価)3,自己他者乖離,前回比較,最初フラグ
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,f64,f64,f64,f64,f64,str,str,str,str,…,str,str,u32,str,u32,null,null,null,null,null,null,null,null,str,f64,str,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,i8
"""F10000001""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""課題設定""",48.304983,"""C""",58.422463,55.351257,55.392977,null,null,"""未知の窓""","""sample16""","""同僚""","""ー""",…,"""ー""","""ー""",null,"""ー""",null,null,null,null,null,null,null,null,null,"""sample43""",0.8163,"""sample21""",0.7989,"""sample52""",0.704,"""影響力の行使""","""創造性""","""ヴィジョン""","""耐性""","""寛容""","""柔軟性""","""sample32""","""sample22""","""sample44""","""地球市民""","""情熱・宣教力""","""ヴィジョン""","""個人的実行力""","""表現力""","""感情コントロール""",-10.117479,-7.046273,1
"""F10000001""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""解決意向""",55.918952,"""B""",66.458537,42.973581,54.875365,null,null,"""秘密の窓""","""sample16""","""同僚""","""ー""",…,"""ー""","""ー""",null,"""ー""",null,null,null,null,null,null,null,null,null,"""sample43""",0.8163,"""sample21""",0.7989,"""sample52""",0.704,"""影響力の行使""","""創造性""","""ヴィジョン""","""耐性""","""寛容""","""柔軟性""","""sample32""","""sample22""","""sample44""","""地球市民""","""情熱・宣教力""","""ヴィジョン""","""個人的実行力""","""表現力""","""感情コントロール""",-10.539586,12.94537,0
"""F10000001""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""創造性""",37.757001,"""D""",54.763871,38.771653,65.694724,null,null,"""未知の窓""","""sample16""","""同僚""","""ー""",…,"""ー""","""ー""",null,"""ー""",null,null,null,null,null,null,null,null,null,"""sample43""",0.8163,"""sample21""",0.7989,"""sample52""",0.704,"""影響力の行使""","""創造性""","""ヴィジョン""","""耐性""","""寛容""","""柔軟性""","""sample32""","""sample22""","""sample44""","""地球市民""","""情熱・宣教力""","""ヴィジョン""","""個人的実行力""","""表現力""","""感情コントロール""",-17.00687,-1.014652,0
"""F10000001""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""sample41""","""事務""","""課長""","""マーケティング""","""男性""","""20代""","""人間関係重視タイプ""","""未来志向タイプ""","""巻き込み型""","""タスク重視タイプ""","""現実志向タイプ""","""構築型""","""論理的思考""",67.767126,"""B""",55.248773,59.486169,43.529731,null,null,"""盲点の窓""","""sample16""","""同僚""","""ー""",…,"""ー""","""ー""",null,"""ー""",null,null,null,null,null,null,null,null,null,"""sample43""",0.8163,"""sample21""",0.7989,"""sample52""",0.704,"""影響力の行使""","""創造性""","""ヴィジョン""","""